# Compare NLP Techniques: Build Model On word2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

### Create word2vec Vectors

In [3]:
# Train a basic word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [6]:
# Replace the words in each text message with the learned word vector
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train['clean_text']])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test['clean_text']])

C:\Users\weber\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
C:\Users\weber\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [7]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [8]:
# What does the unaveraged version look like?
X_train_vect[0]

array([[-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588],
       [-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       ...,
       [-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253]], dtype=float32)

In [9]:
# What does the averaged version look like?
X_train_vect_avg[0]

array([-3.4756674e-03,  1.4834511e-03,  5.1348167e-03,  7.8211501e-03,
       -3.2048165e-03, -6.7716702e-03,  4.5122323e-03,  7.9090949e-03,
       -4.2243828e-03, -4.6397913e-03,  4.5475126e-03, -4.0191007e-03,
       -4.9233045e-03,  6.7542284e-03, -1.8737242e-03,  1.4718683e-03,
        4.3033678e-03,  3.3696441e-03, -6.6502853e-03, -6.2171784e-03,
        5.5068959e-03,  1.5832457e-03,  7.3507968e-03, -3.0993277e-03,
        6.5013450e-03, -1.1072631e-03, -2.3960136e-03,  5.2702511e-03,
       -5.4190699e-03, -6.4291184e-05, -1.1565160e-03, -2.1781889e-03,
        5.8517698e-03, -6.7287926e-03, -8.4827145e-05, -2.1978990e-04,
        7.6460321e-03, -1.5556264e-03,  3.4972832e-03,  3.4704769e-04,
       -3.2393255e-03,  6.4482098e-04, -8.9036888e-03, -2.9241629e-03,
       -1.1495515e-03,  2.6821801e-03, -2.7158537e-03,  5.5564125e-03,
        3.7198409e-03,  6.5265596e-03, -2.3486493e-03, -5.9728837e-04,
       -2.7074241e-03,  1.7862298e-03,  5.0667669e-03,  2.0834939e-04,
      

### Fit RandomForestClassifier On Top Of Word Vectors

In [10]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [11]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

In [12]:
# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.528 / Recall: 0.257 / Accuracy: 0.871
